<a href="https://colab.research.google.com/github/v-square007/FloraScan/blob/main/FloraScan_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit tensorflow==2.15 pillow opencv-python pyngrok keras==2.15
!pip install googletrans==4.0.0-rc1 streamlit-option-menu

In [ ]:
%%writefile app.py
import streamlit as st
from googletrans import Translator
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import zipfile
import tempfile
import os
import numpy as np
import cv2
translator = Translator()
# ====================== #
# 1. BACKGROUND IMAGE
# ====================== #
BACKGROUND_IMAGE_URL = "https://i.pinimg.com/736x/64/32/e0/6432e013e7f15406f0cc71e462668c78.jpg"
selected_lang = st.selectbox("Choose Language", list(languages.keys()))
target_lang = languages[selected_lang]
# ====================== #
# 2. CLEAN STYLING
# ====================== #
st.set_page_config(
    page_title="FloraScan AI",
    page_icon="🌿",
    layout="wide"
)

st.markdown(f"""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@500;600;700&display=swap');

    .block-container {{
    margin-top: -1rem !important;
}}

    /* Background */
    .stApp {{
        background-image: url("{BACKGROUND_IMAGE_URL}");
        background-size: cover;
        background-attachment: fixed;
    }}

    /* Content Box */
    .main-content {{
        background-color: rgba(255, 255, 255, 0.95);
        border-radius: 15px;
        padding: 2rem;
        box-shadow: 0 8px 32px rgba(0,0,0,0.1);
    }}

    /* Text Styling */
    h1, h2, h3 {{
        color: #1a3e1f !important;
        font-family: 'Poppins', sans-serif;
    }}

    body, p, div {{
        color: #333333 !important;
        font-family: 'Poppins', sans-serif;
    }}

    /* Cards */
    .card {{
        background: #E1EACD !important;
        border-radius: 10px !important;
        padding: 2rem !important;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1) !important;

    }}
    languages = {
    'English': 'en',
    'Hindi': 'hi',
    'Spanish': 'es',
    'French': 'fr',
    'Punjabi': 'pa'
}

</style>
""", unsafe_allow_html=True)

# ====================== #
# 3. MODEL LOADING
# ====================== #
@st.cache_resource
def load_model():
    try:
        ZIP_PATH = "trained_model.zip"
        if not os.path.exists(ZIP_PATH):
            st.error("Model file not found")
            return None

        with tempfile.TemporaryDirectory() as tmp_dir:
            with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
                zip_ref.extractall(tmp_dir)

            model_path = os.path.join(tmp_dir, "trained_model.keras")
            if not os.path.exists(model_path):
                st.error("Model missing in zip")
                return None

            return tf.keras.models.load_model(model_path)

    except Exception as e:
        st.error(f"Model error: {str(e)}")
        return None

model = load_model()
if model is None:
    st.stop()
    def translate(text, target_lang):
      try:
        translated = translator.translate(text, dest=target_lang)
        return translated.text
      except Exception as e:
          return text

# ====================== #
# 4. MAIN APP CONTENT
# ====================== #
st.markdown('<div class="main-content">', unsafe_allow_html=True)
st.title(translate("Welcome to FloraScan", target_lang))
st.write(translate("Upload a plant leaf image to detect diseases.", target_lang))
uploaded_file = st.file_uploader(translate("Upload Image", target_lang))
if uploaded_file:
    st.image(uploaded_file)
    st.success(translate("Image uploaded successfully!", target_lang))
# Title
st.markdown('# 🌱 FloraScan – An AI-driven Plant Disease Classification System for Sustainable Agriculture')
st.markdown("""
<div style="font-size: 1.1rem;">
From Soil to Solution — Smarter Farming with FloraScan.
</div>
""", unsafe_allow_html=True)

st.markdown("---")

# Image Upload and Prediction
uploaded_file = st.file_uploader(
    "Upload leaf image (JPG/PNG)",
    type=["jpg", "png", "jpeg"],
    label_visibility="collapsed"
)

if uploaded_file:
    col1, col2 = st.columns(2)

    with col1:
        st.image(uploaded_file,
               use_container_width=True,
               caption="Your Leaf Image")

    with col2:
        try:
            # Save uploaded file to temporary path
            with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp_file:
                tmp_file.write(uploaded_file.getvalue())
                temp_image_path = tmp_file.name

            # Use your exact model testing pipeline
            img = cv2.imread(temp_image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            image = tf.keras.preprocessing.image.load_img(temp_image_path, target_size=(128,128))
            input_array = tf.keras.preprocessing.image.img_to_array(image)
            input_array = np.array([input_array])  # Convert to batch

            # Debugging output
            st.write("Input array shape:", input_array.shape)
            st.write("Pixel range:", np.min(input_array), "to", np.max(input_array))

            # Make prediction
            prediction = model.predict(input_array)

            # Show raw prediction scores
            st.write("Raw prediction scores:", prediction)
            class_names = [
                'Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

            # Get top prediction
            score = tf.nn.softmax(prediction[0])
            predicted_class = class_names[np.argmax(score)]
            confidence = 100 * np.max(score)

            # Display results
            st.markdown('<div class="card">', unsafe_allow_html=True)
            st.markdown('### Diagnosis Results')

            if "healthy" in predicted_class.lower():
                st.success(f"**🍃 Healthy Plant**  \n")
            else:
                st.error(f"**⚠️ Disease Detected: {predicted_class}**  \n")

            st.markdown('</div>', unsafe_allow_html=True)

            # Clean up temp file
            os.unlink(temp_image_path)

        except Exception as e:
            st.error(f"Error: {str(e)}")
            if 'temp_image_path' in locals():
                os.unlink(temp_image_path)

# Future Features
st.markdown("---")


tab1, tab2, tab3 = st.tabs(["Treatment Guide", "Prevention Tips", "Gardening Assistant"])

with tab1:
    st.markdown("""
    - Custom treatment plans
    - Pesticide recommendations
    - Expert consultation
    """)

with tab2:
    st.markdown("""
    - Disease prevention calendar
    - Soil health monitoring
    - Weather alerts
    """)

with tab3:
    st.markdown("""
    - Plant care reminders
    - Gardening tutorials
    - Community forum
    """)

# Team Section
st.markdown("---")
st.markdown('### 👨‍🌾 Meet The Team')

team_cols = st.columns(4)
team = [
    {"name": "Naitra", "id": "2022A7PS0020U"},
    {"name": "Vitasta", "id": "2022A7PS0903U"},
    {"name": "Alina", "id": "2022A7PS0309U"},
    {"name": "Sanya", "id": "2022A7PS0099U"}
]
translations = {
    'en': {'Welcome': "Welcome", 'Upload': "Upload"},
    'hi': {'Welcome': "स्वागत है", 'Upload': "अपलोड करें"},
    'pa': {'Welcome': "ਜੀ ਆਇਆਂ ਨੂੰ", 'Upload': "ਅੱਪਲੋਡ ਕਰੋ"}
}
for i, member in enumerate(team):
    with team_cols[i]:
        st.markdown(f"""
        <div class="card">
            <h4>{member['name']}</h4>
            <p>{member['id']}</p>
        </div>
        """, unsafe_allow_html=True)
        st.title(t("Welcome", target_lang))
st.button(t("Upload", target_lang))

# ====================== #
# 4. ABOUT US & FAQ
# ====================== #

with st.expander("🌿 About Us", expanded=False):
    st.markdown("""
    <div class="card">
    <h3>About FloraScan</h3>
    <p>
    FloraScan is a student-built AI-powered tool to help farmers, gardeners, and researchers quickly identify plant diseases and healthy crops from leaf images. Our mission is to empower sustainable agriculture through accessible technology, enabling smarter and faster decisions for crop health.
    </p>
    <p>
    Inspired by the challenges faced by smallholder farmers and the promise of AI in agriculture, FloraScan combines deep learning with a simple interface. We are committed to continuous improvement, transparency, and supporting the global farming community.
    </p>
    </div>
    """, unsafe_allow_html=True)

with st.expander("❓ FAQ", expanded=False):
    st.markdown("""
    <div class="card">
    <h3>Frequently Asked Questions</h3>
    <ul>
        <li><b>Is FloraScan free?</b><br>Yes, FloraScan is free for personal and educational use.</li>
        <li><b>What images work best?</b><br>Clear, close-up photos of a single leaf on a plain background yield the best results.</li>
        <li><b>Does it work for all plants?</b><br>FloraScan currently supports 15+ major crops and 30+ diseases. We are expanding coverage regularly.</li>
        <li><b>How accurate is it?</b><br>Our model achieves over 90% accuracy on test data, but results may vary with image quality.</li>
        </ul>
    </div>
    """, unsafe_allow_html=True)
with st.expander("🌿 Common Plant Disease", expanded=False):
    st.markdown("""
    <div class="card">
    <h3>Common Plant Diseases with Symptoms,Causes and Control</h3>
    <p>
    <b><u>1. Leaf Spot Diseases :-</b></u>
       <p>
       <b>
       Symptoms: Brown or black spots on leaves, yellow halos, leaf drop
       </b>
       </p>
       <p>
       <b>
       Causes: Fungi or bacteria; wet, humid conditions
       </b>
       </p>
       <p>
       <b>
       Control: Remove infected leaves, improve air circulation, use fungicides
       </b>
       </p>
    </p>
    <p>
    <b><u>2. Powdery Mildew :-</b></u>
    <p>
    <b>
     Symptoms: White, powdery coating on leaves and stems
    </b>
    </p>
    <p>
    <b>
    Causes: Fungal spores in warm, dry conditions with poor airflow
    </b>
    </p>
    <p>
    <b>
    Control: Use sulfur-based fungicides, avoid overhead watering
    </b>
    </p>
    </p>
    <p>
     <b><u>3. Downy Mildew :-</b></u>
     <p>
     <b>
     Symptoms: Yellow spots on upper leaf surface, gray mold on underside
     </b>
     </p>
     <p>
     <b>
     Causes: Oomycete pathogens in moist environments
     </b>
     </p>
     <p>
     <b>
     Control: Remove affected parts, use copper fungicides
     </b>
     </p>
     </p>
    <p>
     <b><u>4. Rust :-</b></u>
     <p>
     <b>
     Symptoms: Reddish-brown or orange powdery spots on leaves
     </b>
     </p>
     <p>
     <b>
     Causes: Fungal spores, spread by wind or water
     </b>
     </p>
     <p>
     <b>
     Control: Prune infected leaves, apply fungicides
     </b>
     </p>
     </p>
    <p>
      <b><u>5. Root Rot :-</b></u>
      <p>
      <b>
      Symptoms: Wilting, yellowing, mushy roots, stunted growth
      </b>
      </p>
      <p>
      <b>
      Causes: Overwatering, poor drainage
      </b>
      </p>
      <p>
      <b>
      Control: Improve soil drainage, use fungicides, replant in dry soil
      </b>
    </p>
    </p>
    </div>
    """, unsafe_allow_html=True)
with st.expander("🌿 How to contact us?", expanded=False):
    st.markdown("""
    <div class="card">
    <h3>Contact Details:</h3>
    <p>
    To contact us for any query kindly email us at florascan@gmail.com
    </p>
    </div>
    """, unsafe_allow_html=True)


st.markdown('</div>', unsafe_allow_html=True)  # Close main-content

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!ngrok authtoken 2vtZLcwt9S9rR0TIsjIOfNelzUF_24hAvQcZBRoJkYGE4FTap
!streamlit run app.py &>/dev/null&
from pyngrok import ngrok
public_url = ngrok.connect(addr='8501')
print("🌟 Your app is live at:", public_url.public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌟 Your app is live at: https://5f6c-34-135-121-199.ngrok-free.app


In [ ]:


# !pkill ngrok